In [1]:
import pandas as pd
import geopandas as gpd
import glob
import os
import numpy as np
import folium

print("Tüm kütüphaneler başarıyla yüklendi.")

Tüm kütüphaneler başarıyla yüklendi.


In [2]:
# mahalle_geojson klasöründeki tüm .geojson dosyalarını bul
mahalle_dosyalari = glob.glob('mahalle_geojson/*.geojson')

gdf_listesi = []
for dosya in mahalle_dosyalari:
    mahalle_gdf = gpd.read_file(dosya)
    # Dosya adından mahalle adını oluştur
    dosya_adi = os.path.basename(dosya)
    mahalle_adi = dosya_adi.replace('.geojson', '').replace('_', ' ')
    mahalle_gdf['mahalle'] = mahalle_adi
    gdf_listesi.append(mahalle_gdf)

# Tüm mahalle geometrilerini tek bir tabloda birleştir
tum_mahalleler_gdf = gpd.GeoDataFrame(pd.concat(gdf_listesi, ignore_index=True))

# Aynı isimdeki mahalle parçalarını (eğer varsa) tek bir geometri altında topla
mahalleler_birlestirilmis_gdf = tum_mahalleler_gdf.dissolve(by='mahalle', aggfunc='first').reset_index()

print(f"Toplam {len(mahalleler_birlestirilmis_gdf)} adet tekil mahalle sınırı başarıyla işlendi.")

Toplam 57 adet tekil mahalle sınırı başarıyla işlendi.


In [3]:
# Demografi dosyasını yükle
demografi_df = pd.read_csv('yenimahalle_demografi.csv')

# İsimlerdeki tutarsızlıkları (büyük/küçük harf, boşluk vb.) gidermek için standart bir anahtar oluştur
mahalleler_birlestirilmis_gdf['mahalle_key'] = mahalleler_birlestirilmis_gdf['mahalle'].str.strip().str.lower().str.replace(' ', '').str.replace('ı','i').str.replace('ş','s').str.replace('ğ','g').str.replace('ç','c').str.replace('ö','o').str.replace('ü','u')
demografi_df['mahalle_key'] = demografi_df['mahalle'].str.strip().str.lower().str.replace(' ', '').str.replace('ı','i').str.replace('ş','s').str.replace('ğ','g').str.replace('ç','c').str.replace('ö','o').str.replace('ü','u')

# Bu standart anahtar üzerinden iki tabloyu birleştir
analiz_df = mahalleler_birlestirilmis_gdf.merge(demografi_df, on='mahalle_key', how='inner')

# Temizlik yap: fazlalık sütunları at ve mahalle ismini orijinal haliyle koru
analiz_df = analiz_df.rename(columns={'mahalle_x': 'mahalle'})
analiz_df = analiz_df.drop(columns=['mahalle_y', 'mahalle_key'])

print("Coğrafi ve demografik veriler başarıyla birleştirildi.")
analiz_df.info()

Coğrafi ve demografik veriler başarıyla birleştirildi.
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mahalle       57 non-null     object  
 1   geometry      57 non-null     geometry
 2   type          57 non-null     object  
 3   id            57 non-null     int32   
 4   tags          57 non-null     object  
 5   nufus         57 non-null     int64   
 6   yas_profili   50 non-null     object  
 7   gelir_duzeyi  50 non-null     object  
dtypes: geometry(1), int32(1), int64(1), object(5)
memory usage: 3.5+ KB


In [4]:
isletme_dosyalari = glob.glob('*_temiz.csv')
isletme_gdf_dict = {}

for dosya in isletme_dosyalari:
    anahtar = dosya.split('_')[0]
    df = pd.read_csv(dosya)
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.boylam, df.enlem), crs="EPSG:4326")
    isletme_gdf_dict[anahtar] = gdf

print(f"{len(isletme_gdf_dict)} türde işletme verisi yüklendi: {list(isletme_gdf_dict.keys())}")

7 türde işletme verisi yüklendi: ['okul', 'hastane', 'market', 'firin', 'restoran', 'cafe', 'eczane']


In [5]:
analiz_df_4326 = analiz_df.to_crs(crs="EPSG:4326")

for isletme_turu, isletme_gdf in isletme_gdf_dict.items():
    birlesmis = gpd.sjoin(isletme_gdf, analiz_df_4326, how="inner", predicate="within")
    sayim = birlesmis['mahalle'].value_counts()
    sutun_adi = f"{isletme_turu}_sayisi"
    analiz_df[sutun_adi] = analiz_df['mahalle'].map(sayim).fillna(0).astype(int)

print("İşletme sayımları analiz tablosuna eklendi.")
display(analiz_df[['mahalle', 'firin_sayisi', 'eczane_sayisi', 'cafe_sayisi']].head())

İşletme sayımları analiz tablosuna eklendi.


,mahalle,firin_sayisi,eczane_sayisi,cafe_sayisi
0,25 Mart Mahallesi,1,0,0
1,Anadolu Mahallesi,0,0,0
2,Ata Mahallesi,0,1,0
3,Avcılar Mahallesi,0,2,0
4,Aşağı Yahyalar Mahallesi,0,0,0


In [6]:
# --- NİHAİ HÜCRE 6 (Düzeltilmiş Versiyon) ---

# ADIM 1: İlk Fırsat Metriklerini Hesaplama
print("İlk fırsat metrikleri hesaplanıyor...")
import numpy as np

analiz_df['firin_basina_kisi'] = analiz_df['nufus'] / analiz_df['firin_sayisi'].replace(0, np.nan)
analiz_df['eczane_basina_kisi'] = analiz_df['nufus'] / analiz_df['eczane_sayisi'].replace(0, np.nan)
analiz_df['market_basina_kisi'] = analiz_df['nufus'] / analiz_df['market_sayisi'].replace(0, np.nan)
analiz_df['cafe_basina_kisi'] = analiz_df['nufus'] / analiz_df['cafe_sayisi'].replace(0, np.nan)
analiz_df['restoran_basina_kisi'] = analiz_df['nufus'] / analiz_df['restoran_sayisi'].replace(0, np.nan)

print("Hesaplama sonrası boş skorlu mahalle sayısı:")
print(analiz_df[['firin_basina_kisi', 'eczane_basina_kisi']].isnull().sum())


# ADIM 2: Eksik Metrikleri Komşu Ortalamalarıyla Doldurma (B Planı ile)
print("\nEksik veriler komşu ortalamalarıyla veya genel ortalama ile dolduruluyor...")
doldurulacak_metrikler = ['firin_basina_kisi', 'eczane_basina_kisi', 'market_basina_kisi', 'cafe_basina_kisi', 'restoran_basina_kisi']
doldurulmus_analiz_df = analiz_df.copy()

for metrik_sutunu in doldurulacak_metrikler:
    if metrik_sutunu in doldurulmus_analiz_df.columns:
        eksik_indeksler = doldurulmus_analiz_df[doldurulmus_analiz_df[metrik_sutunu].isnull()].index

        # B PLANI İÇİN: Genel ortalamayı önceden hesapla
        genel_ortalama = doldurulmus_analiz_df[metrik_sutunu].mean()

        for index in eksik_indeksler:
            eksik_mahalle_geometri = doldurulmus_analiz_df.loc[index].geometry
            komsu_skorlari = []
            dolu_veriler = doldurulmus_analiz_df[metrik_sutunu].notna()
            
            for i, komsu in doldurulmus_analiz_df[dolu_veriler].iterrows():
                if index != i and eksik_mahalle_geometri.intersects(komsu.geometry):
                    komsu_skorlari.append(komsu[metrik_sutunu])
            
            # A PLANI: Eğer komşu bulunduysa, onların ortalamasını kullan
            if komsu_skorlari:
                ortalama_skor = sum(komsu_skorlari) / len(komsu_skorlari)
                doldurulmus_analiz_df.loc[index, metrik_sutunu] = ortalama_skor
            # B PLANI: Eğer hiç komşu bulunamazsa, genel ortalamayı kullan
            else:
                doldurulmus_analiz_df.loc[index, metrik_sutunu] = genel_ortalama
                print(f"-> '{doldurulmus_analiz_df.loc[index].mahalle}' için komşu bulunamadı, GENEL ORTALAMA ({genel_ortalama:,.2f}) atandı.")

print("\nİşlem tamamlandı! Artık boş metrik kalmadı.")
print(doldurulmus_analiz_df[doldurulacak_metrikler].isnull().sum())

İlk fırsat metrikleri hesaplanıyor...
Hesaplama sonrası boş skorlu mahalle sayısı:
firin_basina_kisi     42
eczane_basina_kisi    10
dtype: int64

Eksik veriler komşu ortalamalarıyla veya genel ortalama ile dolduruluyor...
-> 'Beştepe Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (12,315.55) atandı.
-> 'Kuzey Yıldızı Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (12,315.55) atandı.
-> 'Kuzey Yıldızı Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (7,889.78) atandı.
-> 'Ata Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (7,136.94) atandı.
-> 'Karacakaya Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (7,136.94) atandı.
-> 'Kuzey Yıldızı Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (7,136.94) atandı.
-> 'Ata Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (8,648.75) atandı.
-> 'Karacakaya Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (8,648.75) atandı.
-> 'Kuzey Yıldızı Mahallesi' için komşu bulunamadı, GENEL ORTALAMA (8,648.75) atandı.

İşlem tamamlandı! Artık boş metr

In [7]:
# --- NİHAİ HÜCRE 7 (Ankara Arka Planı ile) ---

# Bu hücre, tüm veriyi kullanarak nihai skorları hesaplar ve katmanlı haritaları çizer.

# --- Veri İşleme ve Skorlama (Bu kısım aynı kalıyor) ---
final_df = doldurulmus_analiz_df.copy()

def normalize_sutun(df, sutun_adi):
    # NaN değerleri 0 olarak doldurarak hesaplamayı garantile
    s = df[sutun_adi].fillna(0)
    min_val = s.min()
    max_val = s.max()
    if min_val == max_val: return pd.Series([0.5] * len(df), index=df.index)
    return (s - min_val) / (max_val - min_val)

final_df['alan_km2'] = final_df.to_crs(epsg=32636).geometry.area / 1_000_000
final_df['nufus_yogunlugu'] = final_df['nufus'] / final_df['alan_km2']

final_df['yogunluk_skoru'] = normalize_sutun(final_df, 'nufus_yogunlugu')
final_df['firin_ihtiyac_skoru'] = normalize_sutun(final_df, 'firin_basina_kisi')
final_df['cafe_ihtiyac_skoru'] = normalize_sutun(final_df, 'cafe_basina_kisi')

yas_puanlari = {'Genç Ağırlıklı / Orta Yaş Ağırlıklı': 1.0, 'Orta Yaş Ağırlıklı / Karma': 0.8, 'Yaşlı Ağırlıklı / Karma': 0.2, 'Orta Yaş Ağırlıklı': 0.6, 'Karma / Orta Yaş Ağırlıklı': 0.7 }
gelir_puanlari = { 'Orta / Orta-Yüksek': 1.0, 'Orta-Düşük / Orta':  0.5, 'Orta': 0.6 }
final_df['yas_skoru_cafe'] = final_df['yas_profili'].map(yas_puanlari).fillna(0.1)
final_df['gelir_skoru'] = final_df['gelir_duzeyi'].map(gelir_puanlari).fillna(0.1)

agirlik_firin = { 'ihtiyac': 0.7, 'yogunluk': 0.3 }
final_df['firsat_skoru_firin'] = (final_df['firin_ihtiyac_skoru'] * agirlik_firin['ihtiyac'] + final_df['yogunluk_skoru'] * agirlik_firin['yogunluk'])
final_df['firsat_skoru_firin'] = (normalize_sutun(final_df, 'firsat_skoru_firin') * 100).round(1)

agirlik_cafe = { 'ihtiyac': 0.4, 'yogunluk': 0.2, 'yas': 0.3, 'gelir': 0.1 }
final_df['firsat_skoru_cafe'] = (final_df['cafe_ihtiyac_skoru'] * agirlik_cafe['ihtiyac'] + final_df['yogunluk_skoru'] * agirlik_cafe['yogunluk'] + final_df['yas_skoru_cafe'] * agirlik_cafe['yas'] + final_df['gelir_skoru'] * agirlik_cafe['gelir'])
final_df['firsat_skoru_cafe'] = (normalize_sutun(final_df, 'firsat_skoru_cafe') * 100).round(1)


# --- YENİ HARİTALAMA BÖLÜMÜ ---

# 1. Ankara il sınırını yükle
ankara_gdf = gpd.read_file("ankara_il_siniri.geojson")

# 2. Haritanın merkezini belirle
map_center = [39.93, 32.85] # Ankara merkezi

# 3. Fırın Fırsat Haritasını Oluştur
m_firin_final = folium.Map(location=map_center, zoom_start=10, tiles="cartodbpositron")

# Arka plan katmanı olarak tüm Ankara'yı açık yeşil yap
folium.GeoJson(
    ankara_gdf,
    style_function=lambda x: {'fillColor': '#d4edda', 'color': '#888888', 'weight': 1, 'fillOpacity': 0.5},
    name="Ankara İli"
).add_to(m_firin_final)

# Analiz katmanı olarak Yenimahalle'yi skorlara göre renklendir
folium.Choropleth(
    geo_data=final_df.to_json(),
    name='Fırın Fırsat Skoru',
    data=final_df,
    columns=['mahalle', 'firsat_skoru_firin'],
    key_on='feature.properties.mahalle',
    fill_color='Oranges',
    fill_opacity=0.8,
    line_opacity=0.5,
    legend_name='Fırın Fırsat Skoru (Yenimahalle)',
    highlight=True
).add_to(m_firin_final)

folium.LayerControl().add_to(m_firin_final)
print("--- FIRIN FIRSAT HARİTASI (Ankara Arka Planı ile) ---")
display(m_firin_final)

--- FIRIN FIRSAT HARİTASI (Ankara Arka Planı ile) ---
